In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 构建简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(1, 1)  # 单层全连接网络
    def forward(self, x):
        return self.fc(x)

# 2. 数据准备
x_train = torch.rand(100, 1)  # 100个训练样本，1个特征
y_train = 2 * x_train + 1     # 假设目标是 y = 2x + 1

# 3. 定义模型、损失函数和优化器
model = SimpleNN()
criterion = nn.MSELoss()  # 均方误差损失
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 4. 训练过程
epochs = 100
for epoch in range(epochs):
    model.train()
    # 前向传播
    output = model(x_train)
    loss = criterion(output, y_train)
    # 反向传播
    optimizer.zero_grad()  # 清空梯度
    loss.backward()        # 计算梯度
    optimizer.step()       # 更新参数
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

# 1. 定义CustomDataset类：返回图片路径和标注路径
class CustomDataset(Dataset):
    def __init__(self, data_dir, annotations_dir):
        self.data_dir = data_dir  # 图片路径
        self.annotations_dir = annotations_dir  # 标注文件路径
        self.data_files = os.listdir(data_dir)  # 获取数据目录下的所有图片文件
        self.annotation_files = os.listdir(annotations_dir)  # 获取标注文件

    def __len__(self):
        return len(self.data_files)  # 数据集的大小

    def __getitem__(self, idx):
        # 返回图片路径和对应的标注文件路径
        img_path = os.path.join(self.data_dir, self.data_files[idx])
        annotation_path = os.path.join(self.annotations_dir, self.annotation_files[idx])
        return img_path, annotation_path


# 2. 定义DataLoaderWithAnnotations类：负责按需加载图片和标注
class DataLoaderWithAnnotations:
    def __init__(self, img_path, annotation_path):
        self.img_path = img_path  # 图片路径
        self.annotation_path = annotation_path  # 标注文件路径

    def load_image(self):
        # 读取图片
        img = Image.open(self.img_path)  # 使用PIL库读取图片
        img = img.convert("RGB")  # 确保图片是RGB格式
        return img

    def load_annotation(self):
        # 读取标注文件（假设标注文件是一个文本文件，且每行包含标注内容）
        with open(self.annotation_path, 'r') as file:
            annotation = file.readlines()  # 读取文件内容并返回
        return annotation


# 3. 定义简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(224*224*3, 512)  # 假设图片大小是224x224x3
        self.fc2 = nn.Linear(512, 1)  # 输出一个标量作为回归值（根据任务的需求进行调整）

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 扁平化处理
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 4. 定义训练函数
def train_model(train_loader, model, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()  # 设置模型为训练模式
        
        running_loss = 0.0
        for img_path, annotation_path in train_loader:
            # 创建DataLoaderWithAnnotations对象
            data_loader = DataLoaderWithAnnotations(img_path, annotation_path)
            
            # 按需加载图片和标注
            image = data_loader.load_image()
            annotation = data_loader.load_annotation()

            # 数据预处理：转换为Tensor，并进行必要的变换（如resize、归一化等）
            image = image.resize((224, 224))  # 假设我们需要将图片调整为224x224大小
            image = torch.tensor(np.array(image), dtype=torch.float32).permute(2, 0, 1) / 255.0  # 转换为Tensor并归一化
            
            # 假设标注是一个数值，进行简单的处理（你可以根据实际任务修改这里）
            annotation = torch.tensor([float(anno.strip()) for anno in annotation], dtype=torch.float32)
            
            # 前向传播
            output = model(image.unsqueeze(0))  # 添加batch维度
            loss = criterion(output, annotation.unsqueeze(0))  # 计算损失

            # 反向传播
            optimizer.zero_grad()  # 清空梯度
            loss.backward()        # 计算梯度
            optimizer.step()       # 更新权重

            running_loss += loss.item()  # 记录损失

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


# 5. 数据准备
data_dir = 'path_to_images'  # 图片存放目录
annotations_dir = 'path_to_annotations'  # 标注文件存放目录

# 创建数据集对象
train_dataset = CustomDataset(data_dir, annotations_dir)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)  # 单样本训练

# 6. 定义模型、损失函数和优化器
model = SimpleNN()
criterion = nn.MSELoss()  # 假设这是一个回归问题，可以根据任务需要修改
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 7. 训练模型
train_model(train_loader, model, criterion, optimizer, epochs=10)